In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder\
.appName('Standardizing_ContactDetails')\
.getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/08/20 14:35:37 WARN Utils: Your hostname, GUNAs-MacBook-Air.local, resolves to a loopback address: 127.0.0.1; using 10.0.0.190 instead (on interface en0)
25/08/20 14:35:37 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/08/20 14:35:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/08/20 14:35:47 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [17]:
df = spark.read.option("header", True).csv("us_customer_data 2.csv")

In [18]:
from pyspark.sql.functions import *

In [20]:
# 3. Cleaning first and last names
df = df.withColumn("name", initcap(trim(col("name"))))

In [21]:
# 4. Standardize phone number: keep only digits, add "+1-" if US
df = df.withColumn("phone", regexp_replace(col("phone"), "[^0-9]", "")) \
       .withColumn("phone", when(col("phone") != "", concat_ws("-", lit("+1"), col("phone")))
                            .otherwise(col("phone")))

In [22]:
df = df.withColumn("email", when(
    (col("email").isNull()) | (col("email") == ""),
    concat_ws("", lower(regexp_replace(col("name"), " ", "")), lit("@company.com"))
).otherwise(lower(trim(col("email")))))

In [23]:
df = df.withColumn("address", trim(col("address")))

In [27]:
final_df = df.select(
    "customer_id", "name", "email", "phone", "address", "registration_date", "loyalty_status")


In [28]:
final_df.show()

+-----------+--------------------+--------------------+--------------------+--------------------+-----------------+--------------+
|customer_id|                name|               email|               phone|             address|registration_date|loyalty_status|
+-----------+--------------------+--------------------+--------------------+--------------------+-----------------+--------------+
|          1|       Michelle Kidd|  vayala@example.net|                NULL|USNS Santiago, FP...|       2025-01-25|          Gold|
|          2|         Brad Newton|taylorcatherine@e...|    +1-7595188536738|38783 Oliver Stre...|       2023-07-13|        Silver|
|          3|        Larry Torres|dsanchez@example.net|+1-00132352530949...|6845 Steele Turnp...|       2023-08-18|        Bronze|
|          4|      Kimberly Price|jessicaknight@exa...|+1-00194763342240...|1631 Alexis Meado...|       2024-12-08|          Gold|
|          5|    Matthew Phillips|qwilliams@example...|+1-00186965056828385|2274 Wi